In [ ]:
import requests
import os
import json
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv()

## Collect the data for the rest of 2016 of **main** cities

In [ ]:
# Set the url
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Settings:
stations = [
            '74486',            # New York - John F. Kennedy Airport
            '72509',            # Boston Logan International Airport
            '72405'#,            # Washington D.C. National Airport
#            '72502',            # New Jersey - Newark Airport
#            '72202',            #'Miami International Airport'
#            '72243',            #'Houston, TX Intercontinental'
#            '72494',            #'San Francisco Airport'
#            '72793'             # Seattle-Tacoma Airport
            ]

start_dates = [
                '2016-03-01'
                ]

end_dates = [
            '2016-11-30'
            ]

headers = {
    "X-RapidAPI-Key": os.getenv('api_key'),
    "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df_3 = pd.DataFrame([])

# Loop through all locations and all dates
for station in stations:
    for start_date, end_date in zip(start_dates, end_dates):
        querystring = {
                        "station": station,
                        "start": start_date,
                        "end": end_date,
                        "units": "metric"
                        }
        
        # Request data from url
        r = requests.get(url, headers=headers, params=querystring)

        time.sleep(5) #uncomment if you run into a query limit

        # Decode response with json decoder
        weather_temp_3 = r.json()

        # Flatten json response
        weather_temp_df_3 = pd.json_normalize(weather_temp_3,
                                             sep='_',
                                             record_path='data',
                                             record_prefix='weather_'
                                             )
        
        # Set the station code for all rows related to the current date range
        weather_temp_df_3['station'] = station

        # Concatenate dataframes
        weather_df_3 = pd.concat([weather_df_3, weather_temp_df_3], ignore_index=True)

# Print final dataset weather_df
weather_df_3

In [ ]:
weather_df_3['station'].unique()

## Collect the data for the rest of 2016 of the **other** cities

In [ ]:
# Creating the DataFrame for the cities to compare


# Set the url
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Settings:
stations = [
#            '74486',  # New York - John F. Kennedy Airport
#            '72509',            # Boston Logan International Airport
#            '72405',            # Washington D.C. National Airport
#            '72502',            # New Jersey - Newark Airport
            '72202',            #'Miami International Airport'
            '72243',            #'Houston, TX Intercontinental'
            '72494',            #'San Francisco Airport'
            '72793'             # Seattle-Tacoma Airport
            ]

start_dates = [
                '2016-03-01'
                ]

end_dates = [
            '2016-11-30'
            ]

headers = {
    "X-RapidAPI-Key": os.getenv('api_key'),
    "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df_4 = pd.DataFrame([])

# Loop through all locations and all dates
for station in stations:
    for start_date, end_date in zip(start_dates, end_dates):
        querystring = {
                        "station": station,
                        "start": start_date,
                        "end": end_date,
                        "units": "metric"
                        }
        
        # Request data from url
        r = requests.get(url, headers=headers, params=querystring)

        time.sleep(5) #uncomment if you run into a query limit

        # Decode response with json decoder
        weather_temp_4 = r.json()

        # Flatten json response
        weather_temp_df_4 = pd.json_normalize(weather_temp_4,
                                             sep='_',
                                             record_path='data',
                                             record_prefix='weather_'
                                             )
        
        # Set the station code for all rows related to the current date range
        weather_temp_df_4['station'] = station

        # Concatenate dataframes
        weather_df_4 = pd.concat([weather_df_4, weather_temp_df_4], ignore_index=True)

# Print final dataset weather_df
weather_df_4

In [ ]:
weather_df_4['station'].unique()

**Don't let the next cell run, because I just copy/pasted the cell from the other notebook, but didn't exactly look over the code**

**And I also think we just need to do this part once!**

In [ ]:
import sys
sys.path.append('../')  # Add the parent folder to the Python path

from sql_functions import get_engine  # Import the function

# Now you can use my_function() in your notebook

In [ ]:
import psycopg2

schema = 'group3'
table_name = f'weather_main_cities_rest_of_2016'

engine = get_engine()

if engine!=None:
    try:
        weather_df_3.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')

In [ ]:
import psycopg2

schema = 'group3'
table_name = f'weather_other_cities_rest_of_2016'

engine = get_engine()

if engine!=None:
    try:
        weather_df_4.to_sql(name=table_name, # Name of SQL table variable
                        con=engine, # Engine or connection
                        schema=schema, # your class schema variable
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print('No engine')